In [ ]:
!pip install tensorflow==2.5.0
!pip install gym
!pip install keras
!pip install keras-rl2

In [1]:
import sys
#sys.path.append('/home/phil/work/private/chess/ml/.venv/lib/python3.9/site-packages')
import gym
import random

In [4]:
import numpy as np

import tensorflow as tf

# optional set GPU if available
config = tf.compat.v1.ConfigProto( device_count = {'GPU': 0 , 'CPU': 2} ) 
sess = tf.compat.v1.Session(config=config) 

import keras

keras.backend.set_session(sess)

2021-07-21 20:06:54.176577: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-07-21 20:06:54.177195: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-07-21 20:06:54.177235: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-07-21 20:06:54.177269: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (cinnamon): /proc/driver/nvidia/version does not exist


AttributeError: module 'keras.utils.generic_utils' has no attribute 'populate_dict_with_module_objects'

In [5]:
# load environment
env = gym.make('CartPole-v0')
states = env.observation_space.shape[0]
actions = env.action_space.n


In [29]:
actions
print(states)

4


In [ ]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action = random.choice([0, 1])
        n_state, reward, done, info = env.step(action)
        score+=reward
    print(f'Episode: {episode} Score: {score}')


In [19]:
def build_model(states, actions):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Flatten(input_shape=(1, states)))
    model.add(tf.keras.layers.Dense(24, activation='relu'))
    model.add(tf.keras.layers.Dense(24, activation='relu'))
    model.add(tf.keras.layers.Dense(actions, activation='linear'))
    return model



In [20]:
model = build_model(states, actions)

In [21]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_4 (Flatten)          (None, 4)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 24)                120       
_________________________________________________________________
dense_4 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 50        
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


In [22]:
# agents for training
from rl.agents import DQNAgent
# policy
from rl.policy import BoltzmannQPolicy
# maintain memory, use sequential with sequential model
from rl.memory import SequentialMemory

In [48]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

from rl.core import Env


# Test of own enviornment
class MyEnv(Env):
    def __init__(self, observation_shape):
        self.observation_shape = observation_shape

    def step(self, action):
        return self._get_obs(), random.choice([0, 1]), random.choice([True, False]), {}

    def reset(self):
        return self._get_obs()

    def _get_obs(self):
        if type(self.observation_shape) is list:
            return [np.random.random(s) for s in self.observation_shape]
        else:
            return np.random.random(self.observation_shape)

    def __del__(self):
        pass

env_2 = MyEnv((states))
print(env_2.observation_shape)
states_2 = env_2.observation_space.shape[0]
actions_2 = env_2.action_space.n
dqn.fit(env_2, nb_steps=10, visualize=False, verbose=1)

4


AttributeError: 'NoneType' object has no attribute 'shape'

In [47]:
dqn = build_agent(model, actions)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
dqn.compile(optimizer=optimizer, metrics=['mae']) #mean absolute error
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 95s 10ms/step - reward: 1.0000
51 episodes - episode_reward: 192.961 [106.000, 200.000] - loss: 10.522 - mae: 44.687 - mean_q: 89.539

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 98s 10ms/step - reward: 1.0000
53 episodes - episode_reward: 188.113 [69.000, 200.000] - loss: 18.624 - mae: 46.977 - mean_q: 93.830

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 99s 10ms/step - reward: 1.0000
52 episodes - episode_reward: 195.673 [100.000, 200.000] - loss: 15.987 - mae: 43.127 - mean_q: 85.964

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 97s 10ms/step - reward: 1.0000
52 episodes - episode_reward: 190.615 [89.000, 200.000] - loss: 18.205 - mae: 43.326 - mean_q: 86.499

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 100s 10ms/step - reward: 1.

In [ ]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200
Episode 11: reward: 200.000, steps: 200
Episode 12: reward: 200.000, steps: 200
Episode 13: reward: 200.000, steps: 200
Episode 14: reward: 200.000, steps: 200
Episode 15: reward: 200.000, steps: 200
Episode 16: reward: 200.000, steps: 200
Episode 17: reward: 200.000, steps: 200
Episode 18: reward: 200.000, steps: 200
Episode 19: reward: 200.000, steps: 200
Episode 20: reward: 200.000, steps: 200
Episode 21: reward: 200.000, steps: 200
Episode 22: reward: 200.000, steps: 200
Episode 23: reward: 200.000, steps: 200
Episode 24: reward: 200.000, steps: 200
Episode 25: reward: 

In [ ]:
# visualization
_ = dqn.test(env, nb_episodes=1, visualize=True)

In [31]:
# save/load to memory

dqn.save_weights('dqn_weights.h5f', overwrite=True)

In [32]:
del model
del dqn
del env

In [36]:
env = gym.make('CartPole-v0')
actions = env.action_space.n
states = env.observation_space.shape[0]
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(tf.keras.optimizers.Adam(learning_rate=1e-3), metrics=['mae'])

In [37]:
dqn.load_weights('dqn_weights.h5f')

In [38]:
_ = dqn.test(env, nb_episodes=100, visualize=True)

Testing for 100 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200


KeyboardInterrupt: 

In [39]:
env.close()